In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Connection details
user = "postgres"
password = "postgres"
host = "db"
port = "5432"
database = "ny_taxi"

# Create connection string
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

# Query the table
df = pd.read_sql("SELECT * FROM taxi_zone_lookup", engine)
df.head()

DatabaseError: Execution failed on sql 'SELECT * FROM taxi_zone_lookup': (psycopg2.errors.UndefinedTable) relation "taxi_zone_lookup" does not exist
LINE 1: SELECT * FROM taxi_zone_lookup
                      ^

[SQL: SELECT * FROM taxi_zone_lookup]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [3]:
datalink1 = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet"
datalink2 = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"

In [4]:
df1= pd.read_parquet(datalink1, engine='fastparquet')
df2= pd.read_csv(datalink2)
november_trips = df1[
    (df1['lpep_pickup_datetime'] >= '2025-11-01') &
    (df1['lpep_pickup_datetime'] < '2025-12-01') &
    (df1['trip_distance'] <= 1)
]

num_short_trips = len(november_trips)
num_short_trips

8007

In [5]:
valid_trips = df1[df1['trip_distance'] < 100]

longest_trip = valid_trips.loc[valid_trips['trip_distance'].idxmax()]

# Extract the pickup day
pickup_day_longest_trip = longest_trip['lpep_pickup_datetime'].date()
pickup_day_longest_trip

datetime.date(2025, 11, 14)

In [6]:
# Filter trips on November 18th, 2025
nov18_trips = df1[
    (df1['lpep_pickup_datetime'] >= '2025-11-18') &
    (df1['lpep_pickup_datetime'] < '2025-11-19')
]

# Group by pickup location and sum total_amount
pickup_totals = nov18_trips.groupby('PULocationID')['total_amount'].sum()

# Find the PULocationID with the largest total_amount
max_pu_location_id = pickup_totals.idxmax()

# Get the pickup zone name
pickup_zone_name = df2[df2['LocationID'] == max_pu_location_id]['Zone'].values[0]
pickup_zone_name


'East Harlem North'

In [7]:
# Filter trips picked up in "East Harlem North" in November 2025
east_harlem_november = df1[
    (df1['lpep_pickup_datetime'] >= '2025-11-01') &
    (df1['lpep_pickup_datetime'] < '2025-12-01') &
    (df1['PULocationID'].isin(df2[df2['Zone'] == 'East Harlem North']['LocationID']))
]

# Find the trip with the largest tip
largest_tip_trip = east_harlem_november.loc[east_harlem_november['tip_amount'].idxmax()]

# Get the drop off zone name
dropoff_zone_id = largest_tip_trip['DOLocationID']
dropoff_zone_name = df2[df2['LocationID'] == dropoff_zone_id]['Zone'].values[0]
dropoff_zone_name

'Yorkville West'